In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/megh/AI/Projects/Grapevine_Leaf_Classification'

In [4]:
#entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath: Path



In [5]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

In [6]:
from box import ConfigBox

In [27]:
class ConfigurationManager:
    def __init__(
            self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH) :
                 
        self.config = read_yaml(config_filepath),
        self.params = read_yaml(params_filepath),
        self.config= ConfigBox(self.config[0]) # configbox is returning tuble so to avoid it we are again applying configbox to the dictionary at index 0
        self.params= ConfigBox(self.params[0])



    def get_prepare_callback_config(self)-> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        print(config)
        create_directories([config.root_dir])

        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([ 
                            Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)  ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
    
        )

        return prepare_callback_config
    


In [28]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [29]:
class PrepareCallback:
    def __init__(self, config : PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}",)
        print(tb_running_log_dir)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    
    @property
    def _create_ckpt_callbacks(self):
        
        print(self.config.checkpoint_model_filepath)

        return tf.keras.callbacks.ModelCheckpoint(
            filepath= f"{self.config.checkpoint_model_filepath}",
            save_best_only=True)
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]


In [30]:
try: 
    config = ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2023-08-31 13:49:26,981: INFO: common: yaml fileconfig/config.yamlloaded Successfully]
[2023-08-31 13:49:26,984: INFO: common: yaml fileparams.yamlloaded Successfully]
{'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}
[2023-08-31 13:49:26,986: INFO: common: created directory at:artifacts/prepare_callbacks]
[2023-08-31 13:49:26,987: INFO: common: created directory at:artifacts/prepare_callbacks/checkpoint_dir]
[2023-08-31 13:49:26,988: INFO: common: created directory at:artifacts/prepare_callbacks/tensorboard_log_dir]
artifacts/prepare_callbacks/tensorboard_log_dir/tb_logs_at_2023-08-31-13-49-26
artifacts/prepare_callbacks/checkpoint_dir/model.h5


In [11]:
tf.__version__

'2.13.0'